# DISCLAIMER

This notebook was constructed with the intention of understanding the concepts of **PCA**, so you will find a walkthrough solution, step-by-step into the method implementation.

I also used the scikit to learn the practical/professional way of doing it and to compare the outputs.

[Importing](#importing) and [Data Standardization](#data-standardization) are common to both analysis, so i recomend reading it before jumping into any analysis.

If you wanna see **only the insights and conclusions** of my data analysis, go to [Data analysis](#data-analysis).

Feel free to jump to each part:
- [Step-by-step soltuion](#step-by-step-solution)
- [Practical/Professional Solution](#practical/professional-solution)
- [Comparison between step-by-step and professsional solutions](#comparison)
- [Data analysis](#data-analysis)


# Objectives

- Understand the concepts of PCA and it's implementation
- Perform a manual and a library-based implementation to fix the concepts but to also know how we can quickly do it (professional way)
- Compare the two implementation
- Perform a data analysis using our PCA: Even though it's a dimensionality reduction method, what insights can we gain from our data through it?

Skill objectives

- Gain experience with VScode and python notebooks
- First contact with Scikit-learn
- Use Git and GitHub
- Improve overall writing in english and README files

# Importing

In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc

from plotly.subplots import make_subplots
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [21]:
df = pd.read_csv('apartments_data_cleaned.csv', index_col=0)
df = df.sort_index(axis=1)
initial_df = df.copy()
df

,area,bathrooms,lat,lon,parking,price,rooms
2,124,3,-23.197917,-45.911362,3,1090000,4
4,104,3,-23.201250,-45.883484,2,590000,3
5,60,2,-23.214223,-45.851209,1,300000,2
6,80,2,-23.203881,-45.902105,2,848500,2
7,65,2,-23.226731,-45.903681,1,515000,2
...,...,...,...,...,...,...,...
1491,54,1,-23.171528,-45.789145,1,215000,2
1493,82,2,-23.198504,-45.912865,2,744000,2
1494,58,2,-23.272102,-45.864272,1,215000,2
1495,82,2,-23.197097,-45.888142,1,578000,3


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 875 entries, 2 to 1497
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   area       875 non-null    int64  
 1   bathrooms  875 non-null    int64  
 2   lat        875 non-null    float64
 3   lon        875 non-null    float64
 4   parking    875 non-null    int64  
 5   price      875 non-null    int64  
 6   rooms      875 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 54.7 KB


In [23]:
df.describe()

,area,bathrooms,lat,lon,parking,price,rooms
count,875.000000,875.000000,875.000000,875.000000,875.000000,8.750000e+02,875.000000
mean,88.354286,2.153143,-23.215407,-45.893337,1.659429,7.429614e+05,2.530286
std,51.396651,1.132765,0.028896,0.049290,1.785560,5.942727e+05,0.762349
min,33.000000,1.000000,-23.285937,-45.954734,1.000000,1.600000e+05,1.000000
25%,57.000000,1.000000,-23.232707,-45.909534,1.000000,3.850000e+05,2.000000
50%,73.000000,2.000000,-23.215742,-45.900956,1.000000,5.350000e+05,2.000000
75%,102.000000,2.000000,-23.200832,-45.888325,2.000000,8.530000e+05,3.000000
max,570.000000,6.000000,-22.892742,-44.961914,50.000000,4.800000e+06,5.000000


# Data Standardization

Since our variables have distinct scales, we will standardize them to mean zero and variance one.

In [24]:
# Standard df
scaler = StandardScaler()
st_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
st_df.describe()

,area,bathrooms,lat,lon,parking,price,rooms
count,8.750000e+02,8.750000e+02,8.750000e+02,8.750000e+02,8.750000e+02,8.750000e+02,8.750000e+02
mean,9.744586e-17,9.338562e-17,5.975461e-14,7.220535e-14,-1.827110e-17,2.639159e-17,-2.030122e-18
std,1.000572e+00,1.000572e+00,1.000572e+00,1.000572e+00,1.000572e+00,1.000572e+00,1.000572e+00
min,-1.077618e+00,-1.018572e+00,-2.442229e+00,-1.246336e+00,-3.695230e-01,-9.815272e-01,-2.008478e+00
25%,-6.103942e-01,-1.018572e+00,-5.990666e-01,-3.287864e-01,-3.695230e-01,-6.026966e-01,-6.959925e-01
50%,-2.989118e-01,-1.352712e-01,-1.162079e-02,-1.546753e-01,-3.695230e-01,-3.501429e-01,-6.959925e-01
75%,2.656500e-01,-1.352712e-01,5.046744e-01,1.017454e-01,1.908455e-01,1.852711e-01,6.164934e-01
max,9.376509e+00,3.397931e+00,1.117276e+01,1.890768e+01,2.708854e+01,6.830802e+00,3.241465e+00


# Step-By-Step Solution

## Pearson correlation matrix

Used to verify if variables are related or not

In [25]:
# Get correlation matix
correlation_matrix = st_df.corr(method='pearson')

# Invert the array so the diagonal of 1 goes from top left to bottom right instead of bottom left to top right (only for plotting purpose)
inverted_array = correlation_matrix.values[::-1]

# Visualize correlation matrix
fig = go.Figure(data=go.Heatmap(
    z=inverted_array,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns.sort_values(ascending=False),
    colorscale=[
        [0, 'red'],
        [0.5, 'white'],
        [1, 'green']
    ],
    zmin=-1,
    zmax=1,
    text=inverted_array,
    texttemplate='%{text:.2f}',
    colorbar=dict(title='Correlation')
))

fig.update_layout(
    title='Correlation Matrix',
    xaxis=dict(side='top'),
    width=600,
    height=600
)

fig.show()

## Bartlett Sphericity Test

Used to compare our correlation matrix with the identity matrix. In other words, we are statistically testing the existance of correlation between at least two variables 

In [26]:
chi_square_value, p_value = calculate_bartlett_sphericity(st_df)
print('X²: ', chi_square_value)
print('p-value: ', p_value)

# Evaluate p-value considerng alpha = 0.05
alpha = .05
if p_value > alpha:
    print("There's NO enough correlation between variables. The pearson correlation matrix is statistically equal to the identity. PCA should NOT be done")
else:
    print("There's enough correlation between variables. The pearson correlation matrix is not statistically equal to the identity. PCA CAN be done")

X²:  3217.918070969879
p-value:  0.0
There's enough correlation between variables. The pearson correlation matrix is not statistically equal to the identity. PCA CAN be done


## Eigenvalues and Eigenvectors

The core of PCA, the eigenvectors will be used to determine the directions that maximize the variance of our data, while the eigenvalues the intensity of each direction.

In other words, the eigenvalues indicate the percentage of variance shared by the original variables in the formation of each factor.

In [27]:
# Get eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(correlation_matrix)

# Order eigenvalues and eigenvectors (descending)
order_index = np.argsort(eigenvalues)[::-1]  # Índices para ordenar autovalores em ordem decrescente
eigenvalues = eigenvalues[order_index]
eigenvectors = eigenvectors[:, order_index].transpose()

In [28]:
# Create an eigenvalues data frame to help visualize information
eigenvalues_df = pd.DataFrame(columns=['value'], data=eigenvalues)
eigenvalues_df['variance %'] = eigenvalues_df['value'] / sum(eigenvalues) * 100
eigenvalues_df['cumulative sum'] = eigenvalues_df['value'].cumsum()
eigenvalues_df['cumulative variance %'] = eigenvalues_df['variance %'].cumsum()

print('Eigenvalues data:')
eigenvalues_df


Eigenvalues data:


,value,variance %,cumulative sum,cumulative variance %
0,3.336102,47.658602,3.336102,47.658602
1,1.502748,21.467821,4.838850,69.126424
2,0.885085,12.644071,5.723935,81.770495
3,0.543480,7.763993,6.267414,89.534488
4,0.388811,5.554438,6.656225,95.088926
5,0.227122,3.244598,6.883347,98.333524
6,0.116653,1.666476,7.000000,100.000000


In [29]:
# Eigenvectors data to visualize information
printable_eigenvectors = pd.DataFrame(eigenvectors, columns=st_df.columns, index=[f'PC{i+1}' for i, value in enumerate(eigenvalues)])

print('Eigenvectors:')
printable_eigenvectors


Eigenvectors:


,area,bathrooms,lat,lon,parking,price,rooms
PC1,-0.502010,-0.497743,0.032391,0.132109,-0.217309,-0.498089,-0.431764
PC2,-0.082906,-0.050987,-0.717986,-0.684647,0.018671,-0.057184,-0.051603
PC3,0.110944,0.097931,-0.003605,-0.060560,-0.974878,0.089817,0.126357
PC4,-0.119071,-0.042983,-0.470081,0.487421,0.009886,-0.310392,0.654964
PC5,-0.242867,-0.008343,0.510336,-0.516634,0.040910,-0.340285,0.544174
PC6,0.528966,-0.818195,0.044826,-0.072094,0.015310,0.106748,0.178652
PC7,0.613055,0.262110,0.002274,-0.021243,0.005918,-0.719054,-0.194759


The first two eigenvalues represents 69% of the total variance. Also, They met the [Kaiser’s rule](https://search.r-project.org/CRAN/refmans/EFAtools/html/KGC.html) (eigenvalues > 1). For this reason we will use 02 components in our analysis

In [30]:
# Calculate number of components using Kaiser's rule
number_of_components = sum(1 for value in eigenvalues if value > 1)

# Get the n biggest eigenvalues and their correspondents eigenvectors
selected_components = eigenvalues[0:number_of_components]
selected_eigenvectors = eigenvectors[0:number_of_components]
printable_selected_eigenvectors = pd.DataFrame(selected_eigenvectors, columns=st_df.columns, index=['PC1', 'PC2'])

# Optional statement to standardize eigenvectors to variance of one
# for index, eigenvector in enumerate(selected_eigenvectors):
#     selected_eigenvectors[index] = selected_eigenvectors[index] / selected_components[index]**(1/2)

print('Number of components: ', number_of_components)
print('Eigenvalues: ', selected_components)
print('Eigenvectors: ')
printable_selected_eigenvectors


Number of components:  2
Eigenvalues:  [3.33610217 1.5027475 ]
Eigenvectors: 


,area,bathrooms,lat,lon,parking,price,rooms
PC1,-0.502010,-0.497743,0.032391,0.132109,-0.217309,-0.498089,-0.431764
PC2,-0.082906,-0.050987,-0.717986,-0.684647,0.018671,-0.057184,-0.051603


## Principal components

Now we can calculate the principal components scores

In [31]:
scores = st_df.dot(selected_eigenvectors.transpose())
scores.columns = [f'PC{i+1}' for i in range(number_of_components)] 
scores.head(10)

,PC1,PC2
0,-2.036558,-0.398911
1,-0.662305,-0.565805
2,1.211025,-0.490603
3,0.308912,-0.114974
4,0.827111,0.520841
5,1.251492,0.578491
6,-6.000504,-0.677395
7,-5.335238,-0.694690
8,-3.935328,-0.330484
9,1.005299,-0.511920


Let's see the correlation matrix of the PCA scores to confirm they are not correlated

In [32]:
# Get correlation matix
new_correlation_matrix = scores.corr(method='pearson')

# Invert the array so the diagonal of 1 goes from top left to bottom right instead of bottom left to top right (only for plotting purpose)
inverted_array = new_correlation_matrix.values[::-1]

# Visualize correlation matrix
fig = go.Figure(data=go.Heatmap(
    z=inverted_array,
    x=new_correlation_matrix.columns,
    y=new_correlation_matrix.columns,
    colorscale=[
        [0, 'red'],
        [0.5, 'white'],
        [1, 'green']
    ],
    zmin=-1,
    zmax=1,
    text=inverted_array,
    texttemplate='%{text:.2f}',
    colorbar=dict(title='Correlation')
))

fig.update_layout(
    title='Correlation Matrix',
    xaxis=dict(side='top'),
    width=600,
    height=600
)

fig.show()

# Practical/Professional Solution


In [33]:
# Practical/Professional abordage for PCA
# Number of components was calculated at "Eigenvalues and Eigenvectors" section
pca = PCA(n_components=number_of_components)
practical_scores = pd.DataFrame(pca.fit_transform(st_df), columns=[f'PC{i+1}' for i in range(number_of_components)])
practical_scores.head(10)

,PC1,PC2
0,2.036558,0.398911
1,0.662305,0.565805
2,-1.211025,0.490603
3,-0.308912,0.114974
4,-0.827111,-0.520841
5,-1.251492,-0.578491
6,6.000504,0.677395
7,5.335238,0.694690
8,3.935328,0.330484
9,-1.005299,0.511920


# Comparison


In [34]:
# Compare step-by-step and practical solution
comparative_df = pd.DataFrame()

for i in range(number_of_components):
    
    comparative_df[f'PC{i+1}_step'] = scores.iloc[:,i]
    comparative_df[f'PC{i+1}_practical'] = practical_scores.iloc[:,i]

comparative_df.head(10)

,PC1_step,PC1_practical,PC2_step,PC2_practical
0,-2.036558,2.036558,-0.398911,0.398911
1,-0.662305,0.662305,-0.565805,0.565805
2,1.211025,-1.211025,-0.490603,0.490603
3,0.308912,-0.308912,-0.114974,0.114974
4,0.827111,-0.827111,0.520841,-0.520841
5,1.251492,-1.251492,0.578491,-0.578491
6,-6.000504,6.000504,-0.677395,0.677395
7,-5.335238,5.335238,-0.694690,0.694690
8,-3.935328,3.935328,-0.330484,0.330484
9,1.005299,-1.005299,-0.511920,0.511920


The values in my implementation are the negative of those in the scikit-learn implementation, which does not affect the interpretation since we are concerned with the magnitude of each component.

# Data Analysis

In this section we will cover the PCA results and the conclusions we can take from it.


## Loading plot

The loading plot is a tool to visualize the correlation between each variable and each principal component. The closer they ate to the circle border, more correlated they are.

The coordinates of each variable in the loading plot are the pearson correlation between the variables and the factor created using PC's. There's a good explanation for why we use this coordinates instead of simply the eigenvectors [here](https://stats.stackexchange.com/questions/104306/what-is-the-difference-between-loadings-and-correlation-loadings-in-pca-and
).

These coordinates can be calculate by multiplying the eigenvectors by the square root of their respectives eigenvalues. 

It also help us see the correlation between variables when looking to the angle between them:
- Angles close to zero = High correlation
- Angles close to 90° = Low correlation

In [35]:
# Components dataframe
components = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(number_of_components)], index=st_df.columns)

# Multiply the loadings by the squareroot of their correspondent eigenvalues to get the loadings related to them 
for i, column in enumerate(components.columns):

    components[f'{column}_loadings'] = components[column] * pca.explained_variance_[i]**(1/2)

components

,PC1,PC2,PC1_loadings,PC2_loadings
area,0.502010,0.082906,0.917446,0.101690
bathrooms,0.497743,0.050987,0.909648,0.062539
lat,-0.032391,0.717986,-0.059196,0.880658
lon,-0.132109,0.684647,-0.241434,0.839765
parking,0.217309,-0.018671,0.397142,-0.022902
price,0.498089,0.057184,0.910279,0.070140
rooms,0.431764,0.051603,0.789067,0.063295


In [36]:
# Loading plot figure creation
fig = make_subplots(rows=1, cols=2, subplot_titles=('Loadings (markers)', 'Loadings (arrows)'))  # Títulos dos subplots

label_positions = ['top center', 'bottom center', 'top center', 'top center', 'top center', 'middle right', 'middle left']

# Circle
theta = np.linspace(0, 2*np.pi, 100)
x = np.cos(theta)
y = np.sin(theta)

circle_trace = go.Scatter(
    x=x,
    y=y,
    mode='lines',
    line=dict(color='black'),
    fill='toself',
    fillcolor='whitesmoke',
    showlegend=False,
    opacity=.5,
    marker=dict(color='lightgrey')
)

fig.add_trace(circle_trace, row=1, col=1)
fig.add_trace(circle_trace, row=1, col=2)

# Origin
origin_trace = go.Scatter(
    mode='markers+text',
    text='[0,0]',
    x=[0],
    y=[0],
    marker=dict(color='black'),
    textposition='bottom center',
    showlegend=False
)

fig.add_trace(origin_trace, row=1, col=1)
fig.add_trace(origin_trace, row=1, col=2)

# Plot arrows and scatters
for i, variable in enumerate(list(components.index.to_list())):   

    # Text
    fig.add_trace(
        go.Scatter(
            mode='markers+text',
            text=variable,
            x=[components.PC1_loadings.to_list()[i]],
            y=[components.PC2_loadings.to_list()[i]],
            textposition=label_positions[i],
            name=variable,
            marker=dict(color=pc.qualitative.Plotly[i])
        ),
        row=1, col=1
    )

    # Arrow
    fig.add_annotation(
        
        # Coordinates reference
        axref="x2",
        ayref="y2",
        
        # Arrow start
        ax=0,
        ay=0,
        
        # Arrow end
        x=components.PC1_loadings.to_list()[i],
        y=components.PC2_loadings.to_list()[i],
        
        # Arrow layout
        showarrow=True,
        arrowsize=1,
        arrowhead=1,
        arrowcolor= pc.qualitative.Plotly[i],
        
        row=1, col=2
    )

# Layout
fig.update_layout(
    title='Loading Plot',
    xaxis=dict(
        title='PC 1',
        range=[-1.1, 1.1],
        zerolinecolor='purple'
    ),
    yaxis=dict(
        title='PC 2',
        range=[-1.1, 1.1],
        zerolinecolor='purple'
    ),
    xaxis2=dict(
        title='PC 1',
        range=[-1.1, 1.1],
        zerolinecolor='purple'
    ),
    yaxis2=dict(
        title='PC 2',
        range=[-1.1, 1.1],
        zerolinecolor='purple'
    ),
    width=1200,
    height=600,
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()

In our plot we can see that the price is highly correlated with area, rooms and bathrooms. They are all also highly correlated with PC1.

Also, latitude and longitude do not seems to be correlated with price. They are more correlated with PC2.

A point we need to pay attention is that the area where the appartment is located **does** affects its price. So lat and lon might not be capturing this nuances, or maybe they are just not as important to this dataset as we expected. To evaluate this, we would need to investigate another way to define apartment's localization, like distance from downtown.

Also interesting is the fact that the number of bathrooms are more correlated with price than the number of rooms.

## PC1 x PC2 plot

In this plot we will see how our data is distributed along PC1 and PC2. We are colloring the points by their prices, to see they are affected by PC1 and PC2.

In [37]:
# Combine df so we can use price as color indicator
combined_df = st_df.join(practical_scores)

fig = go.Figure()

# Observations
fig.add_trace(
    go.Scatter(
        x=combined_df.PC1,
        y=combined_df.PC2,
        mode='markers',
        name='observations',
        marker=dict(
            color=combined_df['price'],
            colorscale='Temps',
            colorbar=dict(title='Price'),
            showscale=False
        )
    )
)

# Plot arrows and scatters
for i, variable in enumerate(list(components.index.to_list())):   

    # multiplyer
    multiplyier = 5
    
    # Text
    fig.add_trace(
        go.Scatter(
            mode='markers+text',
            text=variable,
            x=[components.PC1_loadings.to_list()[i]*multiplyier],
            y=[components.PC2_loadings.to_list()[i]*multiplyier],
            textposition=label_positions[i],
            textfont=dict(color=pc.qualitative.Plotly[i]),
            name=variable,
            marker=dict(color=pc.qualitative.Plotly[i])
        ),
    )

    # Arrow
    fig.add_annotation(
        
        # Coordinates reference
        axref="x",
        ayref="y",
        
        # Arrow start
        ax=0,
        ay=0,
        
        # Arrow end
        x=components.PC1_loadings.to_list()[i]*multiplyier,
        y=components.PC2_loadings.to_list()[i]*multiplyier,
        
        # Arrow layout
        showarrow=True,
        arrowsize=1,
        arrowhead=1,
        arrowcolor= pc.qualitative.Plotly[i],
        
    )

# Layout
fig.update_layout(
    title='Observations Plot',
    width=1000,
    height=800
)

fig.show()

We can see that the PC1 keeps the higest amount of information, as we already knew since the calculation of the eigenvalues.

Also, as inferred from perason correlations and loading plots, prices vary majorly in the PC1 axe.

## Checking latitude and longitude influence

Since we expected latitude and longitude to influence prices, let's plot them and see if there's no correlation.

In [38]:

fig = go.Figure()

# Sort by price so the last marker in the same street will be the highest price 
initial_df = initial_df.sort_values('price', ascending=True)

# Format price
formated_price = initial_df['price'].apply(lambda x: f"{x:,}")
formated_price = formated_price.apply(lambda x: f"R$ {x.replace(',','.')},00")

# Scatter
fig.add_trace(go.Scattermapbox(
    lat=initial_df['lat'],
    lon=initial_df['lon'],
    mode='markers',
    marker=dict(
        size=10,
        color=initial_df['price'],
        colorscale='Temps',
        colorbar=dict(title='Price'),
        # showscale=False
    ),
    text=formated_price
))

# Layout
fig.update_layout(
    mapbox=dict(
        # style="open-street-map",
        style="carto-darkmatter",
        zoom=11,
        center=dict(lat=initial_df['lat'].mean(), lon=initial_df['lon'].mean())
    ),
    margin={"r":0,"t":0,"l":0,"b":0},
)

fig.show()

As we can see, there is a place where the price/m² seems higher, but it really looks like not enough to influence prices. Let's check the influence over price/m².

In [41]:

fig = go.Figure()

# Sort by price so the last marker in the same street will be the highest price 
initial_df = initial_df.sort_values('price', ascending=True)

initial_df['price_per_m'] = initial_df['price'] / initial_df['area']

# Format price
formated_price = initial_df['price'].apply(lambda x: f"{x:,}")
formated_price = formated_price.apply(lambda x: f"R$ {x.replace(',','.')},00")

# Scatter
fig.add_trace(go.Scattermapbox(
    lat=initial_df['lat'],
    lon=initial_df['lon'],
    mode='markers',
    marker=dict(
        size=10,
        color=initial_df['price_per_m'],
        colorscale='Temps',
        colorbar=dict(title='Price'),
        # showscale=False
    ),
    text=formated_price
))

# Layout
fig.update_layout(
    mapbox=dict(
        # style="open-street-map",
        style="carto-darkmatter",
        zoom=11,
        center=dict(lat=initial_df['lat'].mean(), lon=initial_df['lon'].mean())
    ),
    margin={"r":0,"t":0,"l":0,"b":0},
)

fig.show()

Whit price/m² we can see that there is a region where the prices/m² are higher. So if we'd done the PCA with this extra column, it might had a different result. 

# Conclusions

## PCA
- Manual implementation of PCA got the same results of library based one. This means we understood the concepts behind PCA.
- The proccess for implementing PCA with scikit-learn is simple and straight forward.

## Data
- Our apartments data prices shows a high correlation with our PC1, wich has also a high correlation with area, bathrooms and rooms. This information will be very useful when constructing a supervised model.
- The prices per m² column (created in the [Checking latitude and longitude influence](#checking-latitude-and-longitude-influence) section) shows a significant difference that could have changed the outputs of our PCA if we included it, maybe raising the correlation between price and latitude or longitude.   
After this insight, I learned that creating other variables from the ones we already have might produce valuable information from our dataset, so we can think about it before start building our analysis. Let's use computers to compute!

## Evaluating Objectives

- Understand the concepts of PCA and it's implementation ✅
- Perform a manual and a library-based implementation to fix the concepts but to also know how we can quickly do it (professional way) ✅
- Compare the two implementation ✅
- Perform a data analysis using our PCA: Even though it's a dimensionality reduction method, what insights can we gain from our data through it? ✅